In [2]:
import pandas as pd
m = pd.read_csv("data/raw/tcga/tcga_clinical_manifest.csv")
print(m.head(3))
print(m["metastasis_status"].value_counts(dropna=False))


   submitter_id                               case_id    primary_diagnosis  \
0  TCGA-DC-6158  0011a67b-1ba9-4a32-a6b8-7850759a38cf  Adenocarcinoma, NOS   
1  TCGA-F4-6854  01240896-3f3f-4bf9-9799-55c87bfacf36  Adenocarcinoma, NOS   
2  TCGA-DC-5337  016c9c14-4c88-49f5-a11a-dd4bc282f11e  Adenocarcinoma, NOS   

   gender age_at_diagnosis tumor_stage  metastasis_status vital_status  \
0    male            25842     Stage I                0.0         Dead   
1  female            28272   Stage IIA                0.0        Alive   
2    male            25202     Stage I                0.0        Alive   

  days_to_death  
0           334  
1           '--  
2           '--  
metastasis_status
0.0    456
NaN    111
1.0     66
Name: count, dtype: int64


In [3]:
import pandas as pd
m = pd.read_csv("data/raw/tcga/tcga_clinical_manifest.csv")
m.head(50).to_csv("data/processed/clinical/manifest_sample.csv", index=False)


In [4]:
import pandas as pd
from pathlib import Path

m = pd.read_csv("data/raw/tcga/tcga_clinical_manifest.csv")

# clean a few fields for readability (optional)
out = m.copy()
# age_at_diagnosis is in days; convert to years (rounded)
if "age_at_diagnosis" in out.columns:
    out["age_years"] = pd.to_numeric(out["age_at_diagnosis"], errors="coerce")/365.25
# days_to_death: turn '-- into NaN and cast numeric
if "days_to_death" in out.columns:
    out["days_to_death"] = pd.to_numeric(out["days_to_death"].replace({"'--": None, "--": None}), errors="coerce")

cols = [
    "submitter_id","case_id","primary_diagnosis","gender",
    "age_years","tumor_stage","metastasis_status","vital_status","days_to_death"
]
available = [c for c in cols if c in out.columns]
sample = out[available].head(50)  # small, safe preview

Path("data/processed/clinical").mkdir(parents=True, exist_ok=True)
sample.to_csv("data/processed/clinical/manifest_sample.csv", index=False)
print("Wrote data/processed/clinical/manifest_sample.csv with", len(sample), "rows")


Wrote data/processed/clinical/manifest_sample.csv with 50 rows


In [6]:
import os, SimpleITK as sitk
p = r"C:\Users\Negar\Desktop\paper_results\Myself\cr_coad_project\data\raw\tcia"
# find a subfolder with DICOMs
for root, dirs, files in os.walk(p):
    if any(f.lower().endswith(".dcm") for f in files):
        dcm_dir = root
        break

series_ids = sitk.ImageSeriesReader.GetGDCMSeriesIDs(dcm_dir)
files = sitk.ImageSeriesReader.GetGDCMSeriesFileNames(dcm_dir, series_ids[0])
img = sitk.ReadImage(files)
print(img.GetSize(), img.GetSpacing())


(512, 512, 439, 1) (1.0, 1.0, 1.0, 1.0)


In [7]:
import pandas as pd
full = r"C:\Users\Negar\Desktop\paper_results\Myself\cr_coad_project\data\processed\images\index_tcia.csv"
sample = r"C:\Users\Negar\Desktop\paper_results\Myself\cr_coad_project\data\processed\images\index_sample.csv"
df = pd.read_csv(full)
df.sample(min(30, len(df))).to_csv(sample, index=False)
print("saved", sample)


saved C:\Users\Negar\Desktop\paper_results\Myself\cr_coad_project\data\processed\images\index_sample.csv


In [8]:
import pandas as pd, os

clin = r"C:\Users\Negar\Desktop\paper_results\Myself\cr_coad_project\data\raw\tcga\tcga_clinical_manifest.csv"
out  = r"C:\Users\Negar\Desktop\paper_results\Myself\cr_coad_project\data\master\patients_master.csv"
os.makedirs(os.path.dirname(out), exist_ok=True)

m = pd.read_csv(clin)
keep = ["submitter_id","case_id","metastasis_status","tumor_stage","gender","age_at_diagnosis","vital_status","days_to_death"]
keep = [c for c in keep if c in m.columns]
m[keep].to_csv(out, index=False)
print("wrote", out, "rows:", len(m))


wrote C:\Users\Negar\Desktop\paper_results\Myself\cr_coad_project\data\master\patients_master.csv rows: 633


In [9]:
import pandas as pd
from pathlib import Path

patch_root = Path("data/processed/images/patches")
rows = []
for p in patch_root.glob("*/*"):
    if p.is_dir():
        n = len(list(p.glob("*.png")))
        rows.append({"patient_id": p.parent.name, "series_id": p.name, "n_slices": n, "path": str(p)})
df = pd.DataFrame(rows).sort_values(["patient_id","series_id"])
df.to_csv("data/processed/images/index_tcia.csv", index=False)
print(df.head(), "\nSaved:", len(df))

                    patient_id  \
0  Colorectal-Liver-Metastases   
1  Colorectal-Liver-Metastases   
2  Colorectal-Liver-Metastases   

                                           series_id  n_slices  \
0  1.3.6.1.4.1.14519.5.2.1.9203.8273.455038217313...       178   
1  1.3.6.1.4.1.14519.5.2.1.9203.8273.533669585389...       161   
2  1.3.6.1.4.1.14519.5.2.1.9203.8273.998520027522...        51   

                                                path  
0  data\processed\images\patches\Colorectal-Liver...  
1  data\processed\images\patches\Colorectal-Liver...  
2  data\processed\images\patches\Colorectal-Liver...   
Saved: 3


In [10]:
from pathlib import Path
import pandas as pd

patch_root = Path("data/processed/images/patches")
rows = []
for patient_dir in patch_root.iterdir():
    if not patient_dir.is_dir(): 
        continue
    for series_dir in patient_dir.iterdir():
        if not series_dir.is_dir(): 
            continue
        n = len(list(series_dir.glob("*.png")))
        rows.append({
            "patient_id": patient_dir.name,
            "series_id": series_dir.name,
            "n_slices": n,
            "path": str(series_dir)
        })
df = pd.DataFrame(rows).sort_values(["patient_id","series_id"])
df.to_csv("data/processed/images/index_tcia.csv", index=False)
print(df.head(), "\nTotal series:", len(df))


     patient_id                                          series_id  n_slices  \
3  CRLM-CT-1001  1.3.6.1.4.1.14519.5.2.1.9203.8273.533669585389...       161   
4  CRLM-CT-1002  1.3.6.1.4.1.14519.5.2.1.9203.8273.455038217313...       178   
5  CRLM-CT-1003  1.3.6.1.4.1.14519.5.2.1.9203.8273.998520027522...        51   
6  CRLM-CT-1004  1.3.6.1.4.1.14519.5.2.1.9203.8273.254242588659...        41   
7  CRLM-CT-1005  1.3.6.1.4.1.14519.5.2.1.9203.8273.337637593354...       141   

                                                path  
3  data\processed\images\patches\CRLM-CT-1001\1.3...  
4  data\processed\images\patches\CRLM-CT-1002\1.3...  
5  data\processed\images\patches\CRLM-CT-1003\1.3...  
6  data\processed\images\patches\CRLM-CT-1004\1.3...  
7  data\processed\images\patches\CRLM-CT-1005\1.3...   
Total series: 23


In [11]:
import pandas as pd
df = pd.read_csv("data/processed/images/index_tcia.csv")
df.head(30).to_csv("data/processed/images/index_sample.csv", index=False)
print("Wrote data/processed/images/index_sample.csv with", min(30, len(df)), "rows")

Wrote data/processed/images/index_sample.csv with 23 rows


In [12]:
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image
import random

root = Path("data/processed/images/patches")
series_dirs = list(root.rglob("*/*"))  # patient_id/series_id
random.shuffle(series_dirs)

for sd in series_dirs[:3]:  # show 3 series
    imgs = sorted(sd.glob("*.png"))
    if not imgs: 
        continue
    mid = imgs[len(imgs)//2]
    print("Show:", sd)
    plt.imshow(Image.open(mid), cmap="gray")
    plt.axis("off")
    plt.show()


In [13]:
from pathlib import Path
from PIL import Image
import numpy as np
import pandas as pd

patch_root = Path("data/processed/images/patches")
rep_root   = Path("data/processed/images/rep_slices")
rep_root.mkdir(parents=True, exist_ok=True)

rows = []
for patient_dir in patch_root.iterdir():
    if not patient_dir.is_dir(): continue
    for series_dir in patient_dir.iterdir():
        if not series_dir.is_dir(): continue
        pngs = sorted(series_dir.glob("*.png"))
        if not pngs: continue
        # pick max-variance slice
        best, best_var = None, -1.0
        for p in pngs:
            arr = np.array(Image.open(p), dtype=np.float32)
            v = float(arr.var())
            if v > best_var:
                best, best_var = p, v
        out_dir = rep_root / patient_dir.name
        out_dir.mkdir(parents=True, exist_ok=True)
        out_file = out_dir / f"{series_dir.name}.png"
        Image.open(best).save(out_file)
        rows.append({
            "patient_id": patient_dir.name,
            "series_id": series_dir.name,
            "rep_slice": str(out_file),
            "var": best_var
        })

df = pd.DataFrame(rows).sort_values(["patient_id","series_id"])
df.to_csv("data/processed/images/rep_index.csv", index=False)
print("Rep slices:", len(df))


Rep slices: 23


In [14]:
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split

idx = pd.read_csv("data/processed/images/index_tcia.csv")
patients = sorted(idx["patient_id"].unique())

train_p, temp_p = train_test_split(patients, test_size=0.3, random_state=42)
val_p, test_p   = train_test_split(temp_p,  test_size=0.5, random_state=42)

def subset(df, plist): 
    return df[df.patient_id.isin(plist)].copy()

train_df = subset(idx, train_p)
val_df   = subset(idx, val_p)
test_df  = subset(idx, test_p)

Path("data/splits").mkdir(parents=True, exist_ok=True)
train_df.to_csv("data/splits/train_series.csv", index=False)
val_df.to_csv("data/splits/val_series.csv", index=False)
test_df.to_csv("data/splits/test_series.csv", index=False)

print(len(train_df), len(val_df), len(test_df))


16 3 4


In [15]:
import pandas as pd
from pathlib import Path

src = Path("data/processed/images/rep_index.csv")
df = pd.read_csv(src)
n = min(30, len(df))
out = Path("data/processed/images/rep_index_sample.csv")
df.sample(n=n, random_state=42).to_csv(out, index=False)
out, n


(WindowsPath('data/processed/images/rep_index_sample.csv'), 23)

In [16]:
import pandas as pd

clinical_path = "data/processed/clinical/clinical_features.csv"
embed_path    = "data/processed/images/image_embeddings_1280.csv"

clinical_df = pd.read_csv(clinical_path)
embed_df    = pd.read_csv(embed_path)

# Attach series_id from embeddings to the clinical dataframe (row-by-row)
clinical_df["series_id"] = embed_df["series_id"]

# Save the fixed version
clinical_fixed_path = "data/processed/clinical/clinical_features_with_id.csv"
clinical_df.to_csv(clinical_fixed_path, index=False)

print("✅ Added series_id to clinical data and saved to:")
print(clinical_fixed_path)
print("✅ New columns:", clinical_df.columns.tolist())


✅ Added series_id to clinical data and saved to:
data/processed/clinical/clinical_features_with_id.csv
✅ New columns: ['gender', 'age_at_diagnosis', 'tumor_stage', 'metastasis_status', 'series_id']


In [1]:
import tensorflow as tf
print("TensorFlow:", tf.__version__)
print("GPU devices:", tf.config.list_physical_devices('GPU'))

TensorFlow: 2.12.0
GPU devices: []


In [18]:
from pathlib import Path

for p in [
    "data/raw/tcga",
    "data/processed/clinical",
    "data/processed/images",
    "data/splits",
    "results/images",
    "results/fusion",
    "results/report",
]:
    print(p, "✅" if Path(p).exists() else "❌")

data/raw/tcga ✅
data/processed/clinical ✅
data/processed/images ✅
data/splits ✅
results/images ✅
results/fusion ✅
results/report ✅


In [5]:
import os
os.chdir(r"C:\Users\Negar\Desktop\paper_results\Myself\cr_coad_project")

import pandas as pd

# Load both files
clinical = pd.read_csv("data/processed/clinical/clinical_features_with_id.csv")
manifest = pd.read_csv("data/raw/tcga/tcga_clinical_manifest.csv")

print("Clinical columns:", clinical.columns.tolist())
print("Manifest columns:", manifest.columns.tolist())

# --- Step 1: Try to detect patient IDs in clinical file ---
# Example: if your image files or series_id encode 'TCGA-DC-6158' or similar
if "series_id" in clinical.columns:
    # Attempt to create a patient ID prefix if possible
    clinical["patient_code"] = clinical["series_id"].astype(str).str.extract(r"(TCGA-[A-Z0-9\-]+)", expand=False)

# --- Step 2: Merge using submitter_id (TCGA patient code) ---
merged = clinical.merge(
    manifest[["case_id", "submitter_id", "metastasis_status"]],
    left_on="patient_code",   # or "patient_id" if it exists
    right_on="submitter_id",
    how="left"
)

print("✅ Merge complete.")
print("Merged columns:", merged.columns.tolist())
print(merged.head(5))

# --- Step 3: Save result ---
merged.to_csv("data/processed/clinical/clinical_features_with_caseid.csv", index=False)
print("✅ Saved: clinical_features_with_caseid.csv")


Clinical columns: ['gender', 'age_at_diagnosis', 'tumor_stage', 'metastasis_status', 'series_id']
Manifest columns: ['submitter_id', 'case_id', 'primary_diagnosis', 'gender', 'age_at_diagnosis', 'tumor_stage', 'metastasis_status', 'vital_status', 'days_to_death']
✅ Merge complete.
Merged columns: ['gender', 'age_at_diagnosis', 'tumor_stage', 'metastasis_status_x', 'series_id', 'patient_code', 'case_id', 'submitter_id', 'metastasis_status_y']
   gender age_at_diagnosis  tumor_stage  metastasis_status_x  \
0     0.0            25842          1.0                    0   
1     1.0            28272          2.0                    0   
2     0.0            25202          1.0                    0   
3     0.0            26420          2.0                    0   
4     1.0            30316          3.0                    0   

                                           series_id patient_code case_id  \
0  1.3.6.1.4.1.14519.5.2.1.9203.8273.533669585389...          NaN     NaN   
1  1.3.6.1.4.1.

STEP 1: Load all relevant files

📊 all_index.csv: 27822 rows
   Columns: ['patient_id', 'series_id', 'slice_path']

📊 clinical_features_with_id.csv: 633 rows
   Columns: ['gender', 'age_at_diagnosis', 'tumor_stage', 'metastasis_status', 'series_id']

📊 image_embeddings_1280.csv: 23 rows
   Columns: ['series_id', '0', '1', '2', '3', '4', '5', '6', '7', '8']... (total: 1281)

STEP 2: Create series_id → patient_id (case_id) mapping

✅ Unique series_id → patient_id pairs: 324
   Sample mapping:
                                             series_id    patient_id
0    1.3.6.1.4.1.14519.5.2.1.9203.8273.533669585389...  CRLM-CT-1001
161  1.3.6.1.4.1.14519.5.2.1.9203.8273.455038217313...  CRLM-CT-1002
339  1.3.6.1.4.1.14519.5.2.1.9203.8273.998520027522...  CRLM-CT-1003
390  1.3.6.1.4.1.14519.5.2.1.9203.8273.254242588659...  CRLM-CT-1004
431  1.3.6.1.4.1.14519.5.2.1.9203.8273.337637593354...  CRLM-CT-1005
572  1.3.6.1.4.1.14519.5.2.1.9203.8273.286130801613...  CRLM-CT-1006
623  1.3.6.1.4.1.1451


STEP 3: Check for missing mappings

❌ Clinical rows with missing series_id in mapping: 610
   Sample missing entries:
    gender age_at_diagnosis  tumor_stage  metastasis_status series_id
23     1.0            30194          3.0                  0       NaN
24     0.0            27771          2.0                  0       NaN
25     0.0            19802          2.0                  0       NaN
26     0.0            21670          2.0                  0       NaN
27     0.0            24865          2.0                  0       NaN

❌ Embedding rows with missing series_id in mapping: 0

⚠️  Mapping entries not present in clinical: 304
⚠️  Mapping entries not present in embeddings: 304
⚠️  Mapping entries missing from BOTH: 304

📊 Unique series counts:
   Clinical: 20
   Embeddings: 20
   Mapping: 324
   Unique patients: 225



STEP 4: Add case_id (patient_id) to clinical and embeddings

⚠️  Clinical rows missing case_id after merge: 610
⚠️  Embedding rows missing case_id after merge: 0

✅ Updated clinical data shape: (633, 6)
✅ Updated embeddings data shape: (23, 1282)

Clinical columns after merge:
['gender', 'age_at_diagnosis', 'tumor_stage', 'metastasis_status', 'series_id', 'case_id']



STEP 5: Save updated files with case_id
✅ Saved: data/processed/clinical/clinical_features_with_caseid.csv
✅ Saved: data/processed/images/image_embeddings_with_caseid.csv



STEP 6: Final verification and summary

✅ Verification of saved files:
   Clinical file: 633 rows, 6 columns
   Embeddings file: 23 rows, 1282 columns

📋 Clinical columns: ['case_id', 'series_id', 'gender', 'age_at_diagnosis', 'tumor_stage', 'metastasis_status']

📋 Embeddings columns (first 10): ['case_id', 'series_id', '0', '1', '2', '3', '4', '5', '6', '7']

⚠️  Clinical rows with missing case_id: 610
⚠️  Embedding rows with missing case_id: 0

📊 Sample clinical data with case_id:
        case_id                                          series_id  gender  \
0  CRLM-CT-1001  1.3.6.1.4.1.14519.5.2.1.9203.8273.533669585389...     0.0   
1  CRLM-CT-1002  1.3.6.1.4.1.14519.5.2.1.9203.8273.455038217313...     1.0   
2  CRLM-CT-1003  1.3.6.1.4.1.14519.5.2.1.9203.8273.998520027522...     0.0   
3  CRLM-CT-1004  1.3.6.1.4.1.14519.5.2.1.9203.8273.254242588659...     0.0   
4  CRLM-CT-1005  1.3.6.1.4.1.14519.5.2.1.9203.8273.337637593354...     1.0   
5  CRLM-CT-1006  1.3.6.1.4.1.14519.5.2.1.92

In [6]:
merged = merged.rename(columns={"metastasis_status_y": "metastasis_status"})
merged = merged.drop(columns=["metastasis_status_x", "submitter_id"], errors="ignore")


In [2]:
import pandas as pd
from pathlib import Path
import os
os.chdir(r"C:\Users\Negar\Desktop\paper_results\Myself\cr_coad_project")

paths = [
    Path("data/raw/tcia/manifest-5OJ3bOs54237323096513094694/metadata.csv"),
    Path("data/raw/tcia/manifest-1669817128730/metadata.csv"),
    Path("data/raw/tcia/manifest-Jayuvect628517908565772204/metadata.csv")
]

dfs = []
for p in paths:
    if p.exists():
        df = pd.read_csv(p)
        print(f"Loaded {len(df)} rows from {p.name}")
        
        # Normalize column names
        df.columns = df.columns.str.strip().str.lower()
        
        # Rename Series UID → series_id
        for col in ["series uid", "series instance uid", "seriesuid"]:
            if col in df.columns:
                df = df.rename(columns={col: "series_id"})
        
        # Rename Subject ID → patient_id
        for col in ["subject id", "subjectid"]:
            if col in df.columns:
                df = df.rename(columns={col: "patient_id"})
        
        # Keep only what we need
        if "series_id" in df.columns and "patient_id" in df.columns:
            dfs.append(df[["series_id", "patient_id"]])
        else:
            print(f"⚠️ Skipped {p.name} — missing expected columns.")

# Combine all metadata into one table
if dfs:
    merged = pd.concat(dfs, ignore_index=True).drop_duplicates()
    out_path = Path("data/raw/tcia/tcia_series_patient_map.csv")
    merged.to_csv(out_path, index=False)
    print(f"✅ Combined mapping saved to: {out_path}")
    print(merged.head())
else:
    print("❌ No valid metadata files found.")


Loaded 34 rows from metadata.csv
Loaded 394 rows from metadata.csv
Loaded 93 rows from metadata.csv
✅ Combined mapping saved to: data\raw\tcia\tcia_series_patient_map.csv
                                           series_id    patient_id
0  1.3.6.1.4.1.14519.5.2.1.8421.4018.116333616240...  TCGA-BM-6198
1  1.3.6.1.4.1.14519.5.2.1.8421.4018.155661808279...  TCGA-BM-6198
2  1.3.6.1.4.1.14519.5.2.1.8421.4018.164159958994...  TCGA-BM-6198
3  1.3.6.1.4.1.14519.5.2.1.8421.4018.115796396159...  TCGA-BM-6198
4  1.3.6.1.4.1.14519.5.2.1.8421.4018.171278458682...  TCGA-BM-6198


In [3]:
df = pd.read_csv("data/raw/tcia/tcia_series_patient_map.csv")
print(df.head())
print(len(df))


                                           series_id    patient_id
0  1.3.6.1.4.1.14519.5.2.1.8421.4018.116333616240...  TCGA-BM-6198
1  1.3.6.1.4.1.14519.5.2.1.8421.4018.155661808279...  TCGA-BM-6198
2  1.3.6.1.4.1.14519.5.2.1.8421.4018.164159958994...  TCGA-BM-6198
3  1.3.6.1.4.1.14519.5.2.1.8421.4018.115796396159...  TCGA-BM-6198
4  1.3.6.1.4.1.14519.5.2.1.8421.4018.171278458682...  TCGA-BM-6198
521


In [4]:
import pandas as pd
from pathlib import Path
import os
os.chdir(r"C:\Users\Negar\Desktop\paper_results\Myself\cr_coad_project")

# Paths
clinical_path = Path("data/processed/clinical/clinical_features_with_id.csv")
mapping_path = Path("data/raw/tcia/tcia_series_patient_map.csv")

# Load data
clinical = pd.read_csv(clinical_path)
mapping = pd.read_csv(mapping_path)

print("Clinical columns:", clinical.columns.tolist())
print("Mapping columns:", mapping.columns.tolist())

# Normalize column names
clinical.columns = clinical.columns.str.strip().str.lower()
mapping.columns = mapping.columns.str.strip().str.lower()

# Check if patient_id is in both
if "patient_id" not in clinical.columns:
    # Some clinical files use 'submitter_id' or 'case_id'
    for alt in ["submitter_id", "case_id"]:
        if alt in clinical.columns:
            clinical = clinical.rename(columns={alt: "patient_id"})

# Merge
merged = pd.merge(clinical, mapping, on="patient_id", how="left")

print("✅ Merged shape:", merged.shape)
print(merged.head())

# Save
out_path = Path("data/processed/clinical/clinical_features_with_caseid_and_series.csv")
merged.to_csv(out_path, index=False)
print(f"✅ Saved merged file: {out_path}")


Clinical columns: ['gender', 'age_at_diagnosis', 'tumor_stage', 'metastasis_status', 'series_id']
Mapping columns: ['series_id', 'patient_id']


KeyError: 'patient_id'